# 🤗 Transformers on AWS Lambda container

学習したモデルはコンテナを使ってAWS Lambda上で推論することも可能です。ここではその方法を紹介します。

**このNotebookではモデルの学習は行わず、HuggingFace 🤗 [Hub](https://huggingface.co/models)にあるモデルを使用します。**

## IAM Role

Note: IAMロールに以下の権限があることを確認してください:

- AmazonSageMakerFullAccess
- AmazonS3FullAccess
- AmazonEC2ContainerRegistryFullAccess
- AWSLambda_FullAccess

ECRとLambdaを使用するために、`AmazonEC2ContainerRegistryFullAccess`と`AWSLambda_FullAccess`が必要になります。

In [ ]:
!pip install "sagemaker>=2.48.0" "torch>=1.7.1" "transformers[ja]==4.6.0" "datasets==1.11" --upgrade

In [ ]:
import os

OUTPUT_DIR = './lambda-docker/model'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

コンテナデプロイのためにModelとTokenizerをダウンロードします（あらかじめ作成したModelとTokenizerを使用する場合は`OUTPUT_DIR`に配置してください）。    
このサンプルでは、🤗 Hubから https://huggingface.co/abhishek/autonlp-japanese-sentiment-59363 をダウンロードします。

In [ ]:
# https://github.com/philschmid/serverless-bert-huggingface-aws-lambda-docker/blob/main/get_model.py
from transformers import AutoModelForSequenceClassification, AutoTokenizer

def get_model(model):
    """Loads model from Hugginface model hub"""
    try:
        model = AutoModelForSequenceClassification.from_pretrained(model)
        model.save_pretrained(OUTPUT_DIR)
    except Exception as e:
        raise(e)

def get_tokenizer(tokenizer):
    """Loads tokenizer from Hugginface model hub"""
    try:
        tokenizer = AutoTokenizer.from_pretrained(tokenizer)
        tokenizer.save_pretrained(OUTPUT_DIR)
    except Exception as e:
        raise(e)

get_model('abhishek/autonlp-japanese-sentiment-59363')
get_tokenizer('abhishek/autonlp-japanese-sentiment-59363')

以下のセルはLambdaで実行するコードを定義して書き出します。

In [ ]:
%%writefile ./lambda-docker/handler.py
import json
import numpy as np
import torch
import torch.nn.functional as F
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig


def encode(tokenizer, inputs):
    """encodes the question and context with a given tokenizer"""
    encoded = tokenizer.encode_plus(inputs)
    return encoded["input_ids"], encoded["attention_mask"]


def serverless_pipeline(model_path='./model'):
    """Initializes the model and tokenzier and returns a predict function that ca be used as pipeline"""
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    labels = ['LABEL_0', 'LABEL_1']
    def predict(inputs):
        """predicts the answer on an given question and context. Uses encode and decode method from above"""
        input_ids, attention_mask = encode(tokenizer, inputs)
        outputs = model(torch.tensor([input_ids]), attention_mask=torch.tensor([attention_mask]))[0]
        prob = F.softmax(outputs, dim=1).detach().numpy().astype(np.float64).max()
        return {'label': labels[torch.argmax(outputs)], 'score': prob}
    return predict


# initializes the pipeline
sequence_classification_pipeline = serverless_pipeline()


def handler(event, context):
    try:
        print(event)
        print(context)
        # loads the incoming event into a dictonary
        body = json.loads(event['body'])
        # uses the pipeline to predict the answer
        output = sequence_classification_pipeline(inputs=body['inputs'])
        print(output)
        return {
            "statusCode": 200,
            "headers": {
                'Content-Type': 'application/json',
                'Access-Control-Allow-Origin': '*',
                "Access-Control-Allow-Credentials": True
            },
            "body": json.dumps(output)
        }
    except Exception as e:
        print(repr(e))
        return {
            "statusCode": 500,
            "headers": {
                'Content-Type': 'application/json',
                'Access-Control-Allow-Origin': '*',
                "Access-Control-Allow-Credentials": True
            },
            "body": json.dumps({"error": repr(e)})
        }

## Create an Amazon ECR registry

`lambda-docker`ディレクトリ内にある`requirements.txt`と`dockerfile`と`handler.py`、Model、TokenizerをECRへアップロードします。

In [ ]:
import boto3

sess = boto3.Session()

registry_name = 'huggingface-lambda-classification'
account = boto3.client('sts').get_caller_identity().get('Account')
region = sess.region_name

!aws ecr create-repository --repository-name {registry_name}

In [ ]:
%%time

image_label = 'v1'
image = f'{account}.dkr.ecr.{region}.amazonaws.com/{registry_name}:{image_label}'

%cd lambda-docker
!docker build -t {registry_name}:{image_label} .
!$(aws ecr get-login --no-include-email --region {region})
!docker tag {registry_name}:{image_label} {image}
!docker push {image}
%cd ../

## Create a Lambda Function

[IAM コンソールでの実行ロールの作成](https://docs.aws.amazon.com/ja_jp/lambda/latest/dg/lambda-intro-execution-role.html)を参照して、
IAM コンソールで実行ロールを作成してください。

1. IAM コンソールの [Roles (ロール)] ページを開きます。
2. [ロールの作成] を選択します。
3. [一般的なユースケース] で、[Lambda] を選択します。
4. [Next: Permissions (次へ: アクセス許可)] を選択します。
5. [アクセス許可ポリシーのアタッチ] で、[AWSLambdaBasicExecutionRole] および [AWSXRayDaemonWriteAccess] AWS 管理ポリシーを選択します。
6. [次へ: タグ] を選択します。
7. [Next: Review] を選択します。
8. [ロール名] に「lambda-role」と入力します（ロール名は任意の名前で構いません）。
9. [ロールの作成] を選択します。

In [ ]:
role = 'YOUR-LAMBDA-ROLE-ARN'
fname = 'bert-lambda-classification'

In [ ]:
lambdac = boto3.client('lambda')

dic = {
    'FunctionName': fname,
    'Role': role,
    'Code':{'ImageUri': image},
    'Timeout': 60,
    'MemorySize': 5120,
    'PackageType':'Image'   #add this parameter
}

lambdac.create_function(**dic)

In [ ]:
# サンプルテキストで試しに推論してみましょう
payload = {
  "body": '{"inputs": "ハワイアンの心和む音楽の中、ちょっとシリアスなドラマが展開していきます。音楽の力ってすごいな、って思いました。"}'
}

In [ ]:
# Lambda関数の作成が完了していない場合、エラーが発生することがあります
# 初回の実行には少し時間がかかります
import json

output = lambdac.invoke(
    FunctionName=fname,
    Payload= json.dumps(payload),
)

In [ ]:
json.loads(output['Payload'].read())